In [ ]:
# Importer les bibliothèques nécessaires pour la modélisation
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Initialisation des modèles
models = {
    'Régression Linéaire': LinearRegression(),
    'Arbre de Décision': DecisionTreeRegressor(random_state=42),
    'Forêt Aléatoire': RandomForestRegressor(random_state=42),
    'SVR': SVR()
}

# 2. Entraînement et évaluation des modèles de régression
results = {}
for model_name, model in models.items():
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)
    
    # Calcul des métriques de performance
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # Sauvegarder les résultats
    results[model_name] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R²': r2
    }

# Convertir les résultats en DataFrame pour plus de lisibilité
results_df = pd.DataFrame(results).T
print("\nÉvaluation des modèles :")
print(results_df)

# 3. Visualisation des performances des modèles
results_df.plot(kind='bar', figsize=(10,6))
plt.title("Comparaison des performances des modèles de régression")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.show()

# 4. Validation croisée pour évaluer la robustesse des modèles
cv_results = {}
for model_name, model in models.items():
    cv_score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_results[model_name] = np.mean(np.abs(cv_score))

cv_results_df = pd.DataFrame(cv_results, index=['MSE'].T)
cv_results_df.plot(kind='bar', figsize=(10,6), color='orange')
plt.title("Validation Croisée des Modèles")
plt.ylabel("MSE (Validation croisée)")
plt.xticks(rotation=45)
plt.show()

# 5. Recherche d'hyperparamètres pour le modèle RandomForest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le score
print("\nMeilleurs paramètres pour Random Forest :")
print(grid_search_rf.best_params_)
print(f"Meilleur score de validation (MSE) : {-grid_search_rf.best_score_}")

# 6. Prédictions et évaluation du modèle optimisé RandomForest
best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

# Calcul des métriques
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Afficher les résultats
print("\nÉvaluation du modèle Random Forest optimisé :")
print(f"MAE : {mae_rf}")
print(f"MSE : {mse_rf}")
print(f"RMSE : {rmse_rf}")
print(f"R² : {r2_rf}")
